

 <h1>
Welcome to the Math Question Answer Verification Competition! 🚀

The goal is to fine-tune a Llama-3-8B model to predict if a given solution to a math problem is correct or not. Your model should output True if the solution is correct, and False otherwise.

This notebook is a starter guide designed to get you up and running quickly. We'll walk through a simplified training process using a small subset of the data (5,000 examples) and lightweight parameters. The main goal here is to understand the complete workflow, from loading data to generating a submission file, not to achieve a top score.

Good luck, and have fun! 🎉

## **Step 1: Install Necessary Libraries**

First, we need to install the required Python libraries. We'll be using the unsloth library, which provides highly efficient, memory-saving training methods for large language models, making it possible to fine-tune powerful models on a single free-tier GPU. We'll also install xformers for further optimization.


In [ ]:
# %%capture
# !pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git@bbdab300de3eb76a435999e92815de452560e51d"
# !pip install --no-deps "xformers<0.0.26" "trl<0.9.0" "peft<0.12.0" "accelerate<0.32.0" "bitsandbytes<0.44.0" "transformers<4.43.0"

  Cloning https://github.com/unslothai/unsloth.git (to revision bbdab300de3eb76a435999e92815de452560e51d) to /tmp/pip-install-roey_ov8/unsloth_59cb6c46acf042be8f821286ae73a2c1
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-roey_ov8/unsloth_59cb6c46acf042be8f821286ae73a2c1
  Running command git rev-parse -q --verify 'sha^bbdab300de3eb76a435999e92815de452560e51d'
  Running command git fetch -q https://github.com/unslothai/unsloth.git bbdab300de3eb76a435999e92815de452560e51d
  Running command git checkout -q bbdab300de3eb76a435999e92815de452560e51d
  Resolved https://github.com/unslothai/unsloth.git to commit bbdab300de3eb76a435999e92815de452560e51d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 45.2 MB

## **Step 2: Load the Model and Tokenizer**

Next, we'll load the Llama-3-8B model, which is the only model permitted for this competition. We'll use Unsloth's FastLanguageModel to handle this efficiently.

A key technique we'll use is 4-bit quantization (load_in_4bit = True). Think of this as compressing the model's knowledge into a much smaller file size. This significantly reduces the amount of GPU memory required, allowing us to fine-tune this large model even on a free platform like Google Colab.



In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048  # Choose any sequence length
dtype = None  # This will auto-detect the best data type for your GPU
load_in_4bit = True  # Use 4-bit quantization to save memory

# Load the model and tokenizer from Hugging Face
# Note: We use the base model, not a 4-bit pre-quantized one,
# to ensure we start from the official weights.
# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "unsloth/Meta-Llama-3.1-8B", # Competition-approved model
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit,
# )

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


🦥 Unsloth Zoo will now patch everything to make training faster!


## **Step 3: Prepare the Dataset**

This is a crucial step where we format our data into a structure the model can learn from. The process involves three parts:

1.  **Loading**: We'll load the official competition dataset from Hugging Face.
2.  **Splitting**: The full dataset is massive. For this starter notebook, we'll create a much smaller, more manageable version to speed things up: **5,000 samples for training** and **500 for validation**.
3.  **Prompting**: We will format each data sample into a clear instructional prompt. This helps the model understand its role as a mathematician verifying a solution.



In [ ]:
from datasets import load_dataset

# Load the full training dataset
full_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="train")

# Shuffle the dataset for randomness and create our smaller splits
shuffled_dataset = full_dataset.shuffle(seed=42)
train_dataset = shuffled_dataset.select(range(40000, 60000))      # Use the first 5,000 for training
validation_dataset = shuffled_dataset.select(range(60000, 65000)) # Use the next 500 for validation

In [ ]:
# The instructional prompt template for training
training_prompt = """You are a great mathematician and you are tasked with finding if a solution to a given maths question is correct or not. Your response should be 'True' if the solution is correct, otherwise 'False'. Below is the Question and Solution.
Question:
{}
GivenAnswer:
{}
GivenSolution (optional reasoning):
{}
Output:
{}"""

# We must add an End Of Sequence (EOS) token to tell the model when a completion is finished.
EOS_TOKEN = tokenizer.eos_token

# This function formats our data samples into the prompt template.
def formatting_prompts_func(examples):
    questions = examples["question"]
    answers = examples["answer"]
    solutions = examples["solution"]
    outputs = examples["is_correct"]
    texts = []
    for question, answer, solution, output in zip(questions, answers, solutions, outputs):
        # Format the prompt and add the EOS token
        text = training_prompt.format(question, str(answer), str(solution), str(output)) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts }

# Apply the formatting function to our training dataset
formatted_train_dataset = train_dataset.map(formatting_prompts_func, batched=True)
formatted_validation_dataset = validation_dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
print(train_dataset)

In [ ]:
print(train_dataset[0])

{'question': 'Find the greatest common divisor of $10293$ and $29384$.', 'is_correct': True, 'answer': '1', 'solution': "To find the greatest common divisor (GCD) of two integers, we can use the built-in gcd() function in sympy.\nHere's how we can use it to find the GCD of $10293$ and $29384$:\n<llm-code>\nfrom sympy import gcd\n\n# Define the integers\nnum1 = 10293\nnum2 = 29384\n\n# Find the GCD using sympy\ngcd(num1, num2)\n</llm-code>\n<llm-code-output>\n1\n</llm-code-output>\nSo the greatest common divisor of $10293$ and $29384$ is $\\boxed{1}$."}


In [ ]:
print(train_dataset.features)

{'question': Value('string'), 'is_correct': Value('bool'), 'answer': Value('string'), 'solution': Value('string')}


In [ ]:
print(len(train_dataset), train_dataset.column_names)

In [ ]:
print(formatted_train_dataset[0])
print(formatted_train_dataset)

## **Step 4: Configure LoRA and Set Up the Trainer**

### **LoRA Configuration**

Instead of training the entire model (which has billions of parameters), we'll use a technique called **Lo**w-**R**ank **A**daptation (LoRA). 🎛️

Think of it like this: rather than rewriting an entire textbook, we're just adding small, efficient "sticky notes" (the LoRA adapters) to update the model's knowledge. This is much faster and requires significantly less memory. We'll use a small **rank** (`r = 8`) to keep the training process light and quick for this starter notebook.


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8, # A small rank for lighter training
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16, # A common practice is to set alpha = 2 * r
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
)

Unsloth: Already have LoRA adapters! We shall skip this step.



### **SFTTrainer Setup**

Now we'll set up the `SFTTrainer` (Supervised Fine-tuning Trainer). This is the main tool from the `trl` library that will handle the entire training loop for us. We'll give it our model, tokenizer, dataset, and a set of training instructions, such as the batch size and number of epochs.

We will train for just **one epoch** (a single pass over our 5,000-sample dataset) to keep this demonstration fast.

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formatted_train_dataset,
    eval_dataset = formatted_validation_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    response_template = "Output:",
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 8,
        warmup_ratio = 0.03,
        # max_steps = 60,
        num_train_epochs = 3,
        # learning_rate = 2e-4,
        learning_rate = 5e-5,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/20000 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/5000 [00:00<?, ? examples/s]

## **Step 4 Extra: Grid Training\!**

In [ ]:
import itertools, json, os, torch
from copy import deepcopy
from trl import SFTTrainer
from transformers import TrainingArguments

# grid = {
#     "lora_r": [1, 2, 4, 8, 16],
#     "lora_alpha": [2, 4, 8, 16, 32],
#     "lora_dropout": [0.0, 0.05],
#     "learning_rate": [1e-4, 2e-4],
#     "max_steps": [200],
# }

grid = {
    "lora_r": [2, 4, 8],
    # "lora_alpha": 2 * [2, 4, 8],
    "lora_dropout": [0.0, 0.05],
    "learning_rate": [1e-4, 2e-4],
    "max_steps": [100],
}

def combos(grid):
    keys = list(grid.keys())
    for vals in itertools.product(*[grid[k] for k in keys]):
        yield dict(zip(keys, vals))

def build_model(lora_r, lora_alpha, lora_dropout):
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "unsloth/Meta-Llama-3.1-8B",
        max_seq_length = max_seq_length,
        dtype = None,
        load_in_4bit = load_in_4bit,
        device_map = {"": 0},
    )
    model = FastLanguageModel.get_peft_model(
        model,
        r = lora_r,
        target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                          "gate_proj", "up_proj", "down_proj"],
        lora_alpha = lora_alpha,
        lora_dropout = lora_dropout,
        bias="none",
        use_gradient_checkpointing="unsloth",
        random_state=42,
    )
    return model

In [ ]:
import re
import torch
from tqdm import tqdm

def evaluate_accuracy(model, tokenizer, dataset, prompt_template):
    FastLanguageModel.for_inference(model)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    # pad_id = tokenizer.pad_token_id or tokenizer.eos_token_id

    def parse_bool(text):
        m = re.search(r'^\s*(True|False)\b', text, flags=re.IGNORECASE)
        return bool(m and m.group(1).lower() == "true")

    correct, total = 0, 0
    for ex in tqdm(dataset):
        q = ex["question"]
        a = ex["answer"]
        s = ex["solution"]
        o = bool(ex["is_correct"])

        prompt = prompt_template.format(q, a, s)
        inputs = tokenizer([prompt], return_tensors="pt").to(device)
        # outputs = model.generate(**inputs, max_new_tokens=8, use_cache=True)
        # response = tokenizer.batch_decode(outputs)
        # pred = parse_bool(response)
        with torch.no_grad():
            gen = model.generate(**inputs, max_new_tokens=8, use_cache=True, do_sample=False)

        input_len = inputs["input_ids"].shape[1]
        new_tokens = gen[0][input_len:]
        text = tokenizer.decode(new_tokens, skip_special_tokens=True)

        if "Output:" in text:
            text = text.split("Output:", 1)[-1]

        pred = parse_bool(text)

        correct += int(pred == o)
        total += 1
    acc = correct / total
    print(f"\nValidation Accuracy: {acc:.4f} ({correct}/{total})")
    return acc



In [ ]:
from transformers import AutoTokenizer
model_id = "unsloth/Meta-Llama-3.1-8B"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
EOS_TOKEN = tokenizer.eos_token


In [ ]:
training_prompt = """You are a great mathematician and you are tasked with finding if a solution to a given maths question is correct or not. Your response should be 'True' if the solution is correct, otherwise 'False'. Below is the Question and Solution.
Question:
{}
GivenAnswer:
{}
GivenSolution (optional reasoning):
{}
Output:
{}"""

In [ ]:
inference_prompt = """You are a great mathematician and you are tasked with finding if a solution to a given maths question is correct or not. Your response should be 'True' if the solution is correct, otherwise 'False'. Below is the Question and Solution.
Question:
{}
GivenAnswer:
{}
GivenSolution (optional reasoning):
{}
Output:
"""

In [ ]:
def formatting_prompts_func(examples):
    qs = examples["question"]
    ans = examples["answer"]
    sol = examples["solution"]
    ys  = examples["is_correct"]
    texts = []
    for q, a, s, y in zip(qs, ans, sol, ys):
        texts.append(training_prompt.format(q, str(a), str(s), str(y)) + EOS_TOKEN)
    return {"text": texts}

formatted_train_dataset = train_dataset.map(formatting_prompts_func, batched=True)
formatted_validation_dataset = validation_dataset.map(formatting_prompts_func, batched=True)


In [ ]:
import gc
def free_cuda():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

In [ ]:
best = {"acc": -1.0, "cfg": None, "ckpt_dir": None}
for cfg in combos(grid):

    cfg["lora_alpha"] = 2 * cfg["lora_r"]

    print("\n=== Try config:", cfg, "===")
    model = build_model(cfg["lora_r"], cfg["lora_alpha"], cfg["lora_dropout"])

    trainer = SFTTrainer(
        model = model,
        tokenizer = tokenizer,
        train_dataset = formatted_train_dataset,
        eval_dataset = formatted_validation_dataset,
        dataset_text_field = "text",
        max_seq_length = max_seq_length,
        response_template = "Output:",
        args = TrainingArguments(
            per_device_train_batch_size = 2,
            gradient_accumulation_steps = 4,
            learning_rate = cfg["learning_rate"],
            warmup_steps = 5,
            max_steps = cfg["max_steps"],
            logging_steps = 10,
            fp16 = not torch.cuda.is_bf16_supported(),
            bf16 = torch.cuda.is_bf16_supported(),
            optim = "adamw_8bit",
            weight_decay = 0.01,
            lr_scheduler_type = "linear",
            seed = 42,
            save_strategy = "no",
            output_dir = "outputs/tmp",
            report_to = "none",
        ),
    )
    trainer.train()

    acc = evaluate_accuracy(model, tokenizer, validation_dataset, inference_prompt)
    if acc > best["acc"]:
        best["acc"] = acc
        best["cfg"] = deepcopy(cfg)
        ckpt_dir = f"outputs/best_lora_r{cfg['lora_r']}_a{cfg['lora_alpha']}_d{cfg['lora_dropout']}_lr{cfg['learning_rate']}"
        os.makedirs(ckpt_dir, exist_ok=True)
        model.save_pretrained(ckpt_dir)
        tokenizer.save_pretrained(ckpt_dir)
        best["ckpt_dir"] = ckpt_dir
    del trainer, model
    free_cuda()
print("\n=== Best on VAL ===")
print(json.dumps({"acc": best["acc"], **best["cfg"]}, indent=2))
print("Adapter saved at:", best["ckpt_dir"])



=== Try config: {'lora_r': 1, 'lora_alpha': 2, 'lora_dropout': 0.0, 'learning_rate': 0.0001, 'max_steps': 200} ===
==((====))==  Unsloth 2025.10.11: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.10.11 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.
The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,000 | Num Epochs = 1 | Total steps = 200
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 2,621,440 of 8,032,882,688 (0.03% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.656400
20,1.448500
30,1.174300
40,0.880900
50,0.811300
60,0.784100
70,0.789200
80,0.783100
90,0.713400
100,0.727000


100%|██████████| 500/500 [01:45<00:00,  4.74it/s]


...

100%|██████████| 500/500 [01:50<00:00,  4.54it/s]



Validation Accuracy: 0.7560 (378/500)

=== Try config: {'lora_r': 2, 'lora_alpha': 4, 'lora_dropout': 0.05, 'learning_rate': 0.0001, 'max_steps': 200} ===
==((====))==  Unsloth 2025.10.11: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,000 | Num Epochs = 1 | Total steps = 200
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 5,242,880 of 8,035,504,128 (0.07% trained)


Step,Training Loss
10,1.642100
20,1.300000
30,0.992600
40,0.793600
50,0.760800
60,0.748700
70,0.766700
80,0.766200
90,0.696200


KeyboardInterrupt: 

In [ ]:
print(best)

{'acc': 0.784, 'cfg': {'lora_r': 1, 'lora_alpha': 32, 'lora_dropout': 0.05, 'learning_rate': 0.0002, 'max_steps': 200}, 'ckpt_dir': 'outputs/best_lora_r1_a32_d0.05_lr0.0002'}


## **Step 5: Start Training\!**

Now, we'll call the `train()` function on our `trainer` object. This will kick off the fine-tuning process. Based on our settings, this will run for one full epoch over our 5,000 examples.

Grab a coffee, as this will take a few minutes\! ☕


In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,000 | Num Epochs = 3 | Total steps = 1,875
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 8 x 1) = 32
 "-____-"     Trainable parameters = 20,971,520 of 8,051,232,768 (0.26% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,0.482300
20,0.475600
30,0.482600
40,0.478500
50,0.465400
60,0.479000
70,0.489600
80,0.472200
90,0.486100
100,0.472700


TrainOutput(global_step=1875, training_loss=0.4343602831522624, metrics={'train_runtime': 8717.4335, 'train_samples_per_second': 6.883, 'train_steps_per_second': 0.215, 'total_flos': 1.1997836089523896e+18, 'train_loss': 0.4343602831522624, 'epoch': 3.0})


## **Step 6: Inference and Evaluation**

Now that our model is trained, we need to test it on our validation set. We'll use a slightly different prompt for inference—one where we leave the `Output:` section blank for the model to complete.

Let's test it on a single example from our validation set to see what it predicts.

In [ ]:
# Prepare the model for faster inference
FastLanguageModel.for_inference(model)

# Create the prompt template for inference (no answer included)
inference_prompt = """You are a great mathematician and you are tasked with finding if a solution to a given maths question is correct or not. Your response should be 'True' if the solution is correct, otherwise 'False'. Below is the Question and Solution.
Question:
{}
GivenAnswer:
{}
GivenSolution (optional reasoning):
{}
Output:
"""

# Select a sample from the validation set
example = validation_dataset[10] # You can change the index (e.g., to 1, 2, 50)
question = example["question"]
answer = example["answer"]
solution = example["solution"]

# Format the prompt with the validation data
inputs = tokenizer(
[
    inference_prompt.format(question, str(answer), str(solution))
], return_tensors = "pt").to("cuda")

# Generate the model's response
outputs = model.generate(**inputs, max_new_tokens = 8, use_cache = True)
response = tokenizer.batch_decode(outputs)

# Print the results
print("#### QUESTION ####")
print(question)
print("\n#### SOLUTION ####")
print(solution)
print("\n#### MODEL'S PREDICTION ####")
# We process the output to show only the generated text
print(response[0].split("Output:\n")[1])
print("\n#### CORRECT ANSWER ####")
print(example["is_correct"])

_ = evaluate_accuracy(model, tokenizer, validation_dataset, inference_prompt)


#### QUESTION ####
How many positive three-digit integers less than 500 have at least two digits that are the same?

#### SOLUTION ####
To calculate the answer, we need to enumerate all the positive three-digit integers less than 500, and select those who have at least two identical digits. 
Here is some Python code to calculate this. 
<llm-code>
count = 0
for number in range(100, 500):
    num_str = str(number)
    digits = set()
    for digit in num_str:
        if digit in digits:
            count += 1
            break
        else:
            digits.add(digit)

print(count)
</llm-code>
<llm-code-output>
112
</llm-code-output>
Hence, the answer is \boxed{112}.

#### MODEL'S PREDICTION ####
False<|end_of_text|>

#### CORRECT ANSWER ####
True


100%|██████████| 5000/5000 [18:00<00:00,  4.63it/s]


Validation Accuracy: 0.8914 (4457/5000)


## **Step 7: Generate Submission File**

This is the final step\! We will now run our fine-tuned model on the official `test` dataset.

We will loop through each example in the test set, generate a prediction, and format the results into a CSV file with two columns: `ID` and `is_correct`, as required by the competition.


In [ ]:
import pandas as pd
from tqdm import tqdm

# Load the official test set
test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")
predictions = []

# A simple function to parse 'True' or 'False' from the model's raw output
def parse_output(response_text):
    # Find the text after "Output:"
    output_part = response_text.split("Output:\n")[-1]
    # Check if "True" is in that part, case-insensitively
    if 'true' in output_part.lower():
        return True
    return False

# Loop through the test dataset and generate a prediction for each example
for example in tqdm(test_dataset):
    question = example["question"]
    answer = example["answer"]
    solution = example["solution"]

    # Format the prompt
    prompt = inference_prompt.format(question, str(answer), str(solution))
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    # Generate the prediction
    outputs = model.generate(**inputs, max_new_tokens=8, use_cache=True)
    response_text = tokenizer.batch_decode(outputs)[0]

    # Parse the prediction and add it to our list
    prediction = parse_output(response_text)
    predictions.append(prediction)

# Create the submission DataFrame
submission = pd.DataFrame({
    'ID': range(len(predictions)),
    'is_correct': predictions
})

# Save the DataFrame to a CSV file
submission.to_csv('submission.csv', index=False)

print("\nSubmission file 'submission.csv' created successfully!")
print("You can now download this file and submit it to the Kaggle competition.")

100%|██████████| 10000/10000 [36:05<00:00,  4.62it/s]


Submission file 'submission.csv' created successfully!
You can now download this file and submit it to the Kaggle competition.


# SAVE THE MODEL TO DRIVE AND RUN INFERENCE
Add code to save the model checkpoint to Google Drive, load the model from the checkpoint, and generate the final submission CSV file.

## Mount google drive

### Subtask:
Mount Google Drive to save the model checkpoint.


**Reasoning**:
Mount Google Drive to save the model checkpoint.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Save model checkpoint

### Subtask:
Save the trained model checkpoint to the specified path in Google Drive.


**Reasoning**:
Define the save path and save the model and tokenizer to Google Drive.



In [ ]:
import os

# Define the path to save the model checkpoint in Google Drive
save_path = "/content/drive/MyDrive/DL_midter_contest/llama3_8b_math_verifier_checkpoint"

# Create the directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)

# Save the model and tokenizer
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model checkpoint and tokenizer saved to: {save_path}")

Model checkpoint and tokenizer saved to: /content/drive/MyDrive/DL_midter_contest/llama3_8b_math_verifier_checkpoint


## Load model from checkpoint

### Subtask:
Load the model from the saved checkpoint.


**Reasoning**:
Load the model and tokenizer from the saved checkpoint path in Google Drive and prepare the model for inference.



In [ ]:
# Define the path where the model checkpoint was saved in Google Drive
save_path = "/content/drive/MyDrive/DL_midter_contest/llama3_8b_math_verifier_checkpoint"

# Load the model and tokenizer from the saved path
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = save_path,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Prepare the loaded model for faster inference
# FastLanguageModel.for_inference(model)

# print(f"Model and tokenizer loaded from: {save_path}")

==((====))==  Unsloth 2025.10.8: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.10.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Generate submission file

### Subtask:
Generate the submission CSV file using the loaded model.


**Reasoning**:
Generate the submission CSV file by iterating through the test dataset, generating predictions using the loaded model, and saving the results to a pandas DataFrame.



In [ ]:
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset

# Load the official test set
test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")
predictions = []

# Create the prompt template for inference (no answer included)
inference_prompt = """You are a great mathematician and you are tasked with finding if a solution to a given maths question is correct or not. Your response should be 'True' if the solution is correct, otherwise 'False'. Below is the Question and Solution.
Question:
{}
Solution:
{}
Output:
"""

# A simple function to parse 'True' or 'False' from the model's raw output
def parse_output(response_text):
    # Find the text after "Output:"
    output_part = response_text.split("Output:\n")[-1]
    # Check if "True" is in that part, case-insensitively
    if 'true' in output_part.lower():
        return True
    return False

# Loop through the test dataset and generate a prediction for each example
for example in tqdm(test_dataset):
    question = example["question"]
    solution = example["solution"]

    # Format the prompt
    prompt = inference_prompt.format(question, str(solution))
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    # Generate the prediction
    outputs = model.generate(**inputs, max_new_tokens=8, use_cache=True)
    response_text = tokenizer.batch_decode(outputs)[0]

    # Parse the prediction and add it to our list
    prediction = parse_output(response_text)
    predictions.append(prediction)

# Create the submission DataFrame
submission = pd.DataFrame({
    'ID': range(len(predictions)),
    'is_correct': predictions
})

# Save the DataFrame to a CSV file
submission.to_csv('submission.csv', index=False)

print("\nSubmission file 'submission.csv' created successfully!")
print("You can now download this file and submit it to the Kaggle competition.")